In [1]:
import sys
import os
import cv2
import numpy as np
import torch

from ortools.graph import pywrapgraph

In [2]:
sys.path.append("../0.mc_utils/")

In [3]:
from openpose.op_utils import play_ob25b_on_video,draw_reided_bboxes,draw_kp2d_to_image
from common.geometry_reid import GeometryReID

In [4]:
video_fname = "E:/4.test_videos/basket_1.mp4"
npz_fname = "E:/20190727-area51/basket_1/op25b_x2ds.npz"
# video_fname = "E:/4.test_videos/4.shotput/cap_20190227_110117_c400260.mp4"
# npz_fname = "E:/20190727-area51/cap_20190227_110117_c400260/op25b_x2ds.npz"
# video_fname = "E:/4.test_videos/dance2.mp4"
# npz_fname = "E:/20190727-area51/dance2//op25b_x2ds.npz"

ret_op25b_x2ds = np.load(npz_fname,allow_pickle=True)["op25b"].item()

In [5]:
# play_ob25b_on_video(video_fname,ret_op25b_x2ds,interval=30)

In [9]:
cap = cv2.VideoCapture(video_fname)
fps = cap.get(cv2.CAP_PROP_FPS)
interval = int(1000//fps)
frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cv2.namedWindow("frame_show")

# select_joints = [0,12,14,11,13,6,8,5,7]
select_joints = [0,5,6,11,12]

geom_reid = GeometryReID(max_hp=300,max_cost=1e4)
geom_reid.reset()
for idx in range(frames):
    is_valid,frame = cap.read()
    h,w,c = frame.shape
    if not is_valid:
        break
    if idx in ret_op25b_x2ds:
        x2ds = ret_op25b_x2ds[idx]
        N = x2ds.shape[0]
        
        #koad features and ReID
        reid_masks = (x2ds[:,select_joints,-1:]>0.3)
        reid_x2ds = x2ds[:,select_joints,:2]
#         reid_x2ds = (2*reid_x2ds-h)/(1e-6+w)*1000*reid_masks
        reid_x2ds = (2*reid_x2ds-h)/(1e-6+w)*1e4*reid_masks
        reids = geom_reid(reid_x2ds.reshape(N,-1))
        
        sys.stdout.write("\r>>>reid:{}<<<<<<<<<".format(reids))
        for pid in range(len(reids)):
            hid,cost = reids[pid]
            x2d = x2ds[pid,...,:2]
            draw_reided_bboxes(frame,x2d,">>{}".format(hid))
            draw_kp2d_to_image(frame,x2d)
            
    cv2.putText(frame, 
                "frame:<{}>".format(idx), (10,25), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (0,255,0), 
                1,
                cv2.LINE_AA)
    cv2.imshow("frame_show",frame)
    if cv2.waitKey(interval) & 0xFF==ord('q'):
        break
cv2.destroyAllWindows()
cap.release()

>>>reid:[[0, 8], [1, 3]]<<<<<<<<<<<<<<<<<<<<<<<<